In [2]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as numpy

In [332]:
driver = webdriver.Chrome()

names, prices, ratings_overall, solds= [], [], [], []
ratings, reviews = [], []

url = 'https://www.tokopedia.com/officialinfinix/etalase/note-series'
driver.get(url)

# Scroll to load products
for scroll in range(6):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for x in soup.find_all('div', {'class': 'css-1sn1xa2'}):
    try:
        # Get product name and price
        name = x.find('div', {'class': 'prd_link-product-name css-3um8ox'})
        price = x.find('div', {'class': 'prd_link-product-price css-h66vau'})
        rating_overall = x.find('span', {'class' : 'prd_rating-average-text css-t70v7i'})
        sold = x.find('span', {'class': 'prd_label-integrity css-1sgek4h'})

        names.append(name.get_text() if name else None)
        prices.append(price.get_text() if price else None)
        ratings_overall.append(rating_overall.get_text() if rating_overall else None)
        solds.append(sold.get_text() if sold else None)

        # Navigate to product page
        # link = x.find('a')['href']
        
        # if link:
        #     driver.get(link)
            
        #     # Scroll reviews
        #     for scroll in range(20):
        #         driver.execute_script("window.scrollBy(0,300)")
        #         time.sleep(1)

        #     html = driver.page_source
        #     soup = BeautifulSoup(html, 'html.parser')

        #     # Ratings and reviews
        #     for z in soup.find_all('div', {'class': "css-1k41fl7"}):
        #         rating = z.find('div', {'data-testid': 'icnStarRating'})
        #         review = z.find('span', {'data-testid': 'lblItemUlasan'})
                
        #         ratings.append(rating.get("aria-label") if rating else None)
        #         reviews.append(review.get_text() if review else None)
        #         num_products1.append(pro)
    
    except Exception as e:
        print(f"Error: {e}")
        continue

driver.quit()
print('Scraping complete.')

print(f'length of names : {len(names)}')
print(f'length of prices : {len(prices)}')
print(f'length of ratings overall : {len(ratings_overall)}')
print(f'length of solds : {len(solds)}')

# print(f'length of ratings : {len(ratings)}')
# print(f'length of reviews : {len(reviews)}')

Scraping complete.
length of names : 7
length of prices : 7
length of ratings overall : 7
length of solds : 7


In [333]:
brands, ram, rom= [], [], []

In [ ]:
brands.extend(['infinix' for i in range(7)])
ram.extend(['0' for i in range(7)])
rom.extend(['0' for i in range(7)])

In [133]:
def clean_analyst(data):
    data = data.dropna()
    data['price'] = [number[2:] for number in data.price]
    data['price'] = [number.replace('.', '') for number in data.price]
    data['sold'] = [number.replace('+ terjual', '') for number in data.sold]
    data['sold'] = [number.replace(' terjual', '') for number in data.sold]
    
    for number in data.sold:
        k = ['1rb', '2rb', '3rb', '4rb', '5rb', '6rb', '7rb', '8rb', '9rb', '10rb']
        if number in k:
            data['sold'] = data['sold'].replace(number, number[:1] + '000')
    return data

def clean_review(df):
    df = df.dropna()
    df['rating'] = df['rating'].str.extract(r'bintang (\d+\.?\d*)')
    df['brand'] = ['realme' for i in range(len(df['rating']))]
    df['rating'] = df['rating'].astype(int)
    df['rating_label'] = df['rating'].apply(
        lambda x: 'bad' if x < 3 else ('neutral' if x == 3 else 'good')
    )
    return df

In [335]:
data = pd.DataFrame({'name':names, 'price':prices, 'rating':ratings_overall, 'sold':solds, 'brand':brands, 'ram':ram, 'rom':rom})
data = clean_analyst(data)
data.head()

C:\Users\MyBook Z Series\AppData\Local\Temp\ipykernel_5880\3627191284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = [number[2:] for number in data.price]
C:\Users\MyBook Z Series\AppData\Local\Temp\ipykernel_5880\3627191284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = [number.replace('.', '') for number in data.price]
C:\Users\MyBook Z Series\AppData\Local\Temp\ipykernel_5880\3627191284.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,name,price,rating,sold,brand,ram,rom
0,Infinix Note 40S 8/256GB - Up to 16GB Extended...,2749000,4.9,500,vivo,0,0
1,Infinix Note 40 Pro+ 5G 12/256GB - Up to 24GB ...,3949000,4.9,250,vivo,0,0
3,Infinix Note 40 Pro 5G 8/256GB - Up to 16GB Ex...,3799000,4.9,1000,vivo,0,0
4,Infinix Note 30 8/256GB - Up To 16GB - 120 HZ ...,2599000,4.9,1000,vivo,0,0
5,Infinix Note 30 PRO 8/256GB -Up To 16GB- 108MP...,3199000,4.9,1000,vivo,0,0


In [336]:
extracted_ram = data['name'].str.extract(r'(\d+)/')[0]
extracted_rom = data['name'].str.extract(r'/(\d+)GB')[0]

data.loc[data['ram']=='0', 'ram'] = extracted_ram
data.loc[data['rom']=='0', 'rom'] = extracted_rom

data['ram'] = data['ram'].fillna(data['name'].str.extract(r'(\d+)GB')[0])
data['rom'] = data['rom'].fillna(data['name'].str.extract(r'/(\d+)\)')[0])

In [337]:
data.head()

,name,price,rating,sold,brand,ram,rom
0,Infinix Note 40S 8/256GB - Up to 16GB Extended...,2749000,4.9,500,vivo,8,256
1,Infinix Note 40 Pro+ 5G 12/256GB - Up to 24GB ...,3949000,4.9,250,vivo,12,256
3,Infinix Note 40 Pro 5G 8/256GB - Up to 16GB Ex...,3799000,4.9,1000,vivo,8,256
4,Infinix Note 30 8/256GB - Up To 16GB - 120 HZ ...,2599000,4.9,1000,vivo,8,256
5,Infinix Note 30 PRO 8/256GB -Up To 16GB- 108MP...,3199000,4.9,1000,vivo,8,256


In [268]:
# data1 = pd.DataFrame({'rating':ratings, 'review':reviews})
# data1 = clean_review(data1)

In [338]:
df = pd.read_csv('combined.csv')
df.head()

,name,price,rating,sold,ram,rom,brand
0,Asus Zenfone 10 8/128GB - Starry Blue,8999000,4.8,60,8,128.0,asus
1,Asus Zenfone 10 8/128GB - Aurora Green,8999000,4.9,60,8,128.0,asus
2,Asus Zenfone 10 8/128GB - Eclipse Red,8999000,5.0,70,8,128.0,asus
3,Asus Zenfone 10 8/128GB - Comet White,8999000,4.8,50,8,128.0,asus
4,Asus Zenfone 10 8/128GB - Midnight Black,8999000,4.9,100,8,128.0,asus


In [339]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    255 non-null    object 
 1   price   255 non-null    int64  
 2   rating  255 non-null    float64
 3   sold    255 non-null    int64  
 4   ram     255 non-null    int64  
 5   rom     253 non-null    float64
 6   brand   255 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 14.1+ KB


In [348]:
combined_data = pd.concat([df, data], ignore_index=True)
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    261 non-null    object
 1   price   261 non-null    object
 2   rating  261 non-null    object
 3   sold    261 non-null    object
 4   ram     261 non-null    object
 5   rom     259 non-null    object
 6   brand   261 non-null    object
dtypes: object(7)
memory usage: 14.4+ KB
